# 查看输出hdf5数据结构

In [ ]:
# Y00855ND: 10m 6.5s
!cargo run -- -i Y00855ND.tissue.gem.gz -o Y00855ND.bgef

   Compiling gem2gef v0.1.0 (/Volumes/MacPassport/rust/geftools_rs)

 --> src/gem_reader.rs:3:5
  |
3 | use hdf5::types::VarLenUnicode;
  |     ^^^^^^^^^^^^^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/gem_reader.rs:4:5
  |
4 | use hdf5::H5Type;
  |     ^^^^^^^^^^^^

   --> src/bgef_writer.rs:136:13
    |
136 |         let gene_exp_bin1 = gene_exp.create_group("bin1")?;
    |             ^^^^^^^^^^^^^ help: if this is intentional, prefix it with an underscore: `_gene_exp_bin1`
    |
    = note: `#[warn(unused_variables)]` on by default

  --> src/bgef_writer.rs:32:8
   |
32 | struct BgefWriter {
   |        ^^^^^^^^^^
   |
   = note: `#[warn(dead_code)]` on by default

   --> src/bgef_writer.rs:47:12
    |
 46 | impl BgefWriter {
    | --------------- associated items in this implementation
 47 |     pub fn new(
    |            ^^^
...
134 |     fn write_bgef_gene_exp(self) -> Result<(), Box<dyn std::error::Erro...
    |        ^^^^^^^^^^^^^^^^^^^

   

In [1]:
import os
import h5py
display(os.getcwd())

def print_structure(name, obj):
    if isinstance(obj, h5py.Group):
        print(f"[Group] {name}")
    elif isinstance(obj, h5py.Dataset):
        print(f"[Dataset] {name}  shape={obj.shape}  dtype={obj.dtype}")

path = "out/test.bgef"
path = "Y00855ND.tissue.bgef"
with h5py.File(path, "r") as f:
    f.visititems(print_structure)

'/Volumes/MacPassport/rust/geftools_rs'

[Group] geneExp
[Group] geneExp/bin1
[Dataset] geneExp/bin1/exon  shape=(123192319,)  dtype=uint32
[Dataset] geneExp/bin1/expression  shape=(123192319,)  dtype=[('x', '<i4'), ('y', '<i4'), ('count', '<u4')]
[Dataset] geneExp/bin1/gene  shape=(43520,)  dtype=[('gene_id', 'O'), ('gene_name', 'O'), ('offset', '<u4'), ('count', '<u4')]
[Group] wholeExp
[Dataset] wholeExp/bin1  shape=(17502, 17043)  dtype={'names': ['mid_count', 'gene_count'], 'formats': ['<u4', '<u2'], 'offsets': [0, 4], 'itemsize': 8}
[Group] wholeExpExon
[Dataset] wholeExpExon/bin1  shape=(17502, 17043)  dtype=uint32


In [2]:
import numpy as np

path = "out/test.bgef"
path = "Y00855ND.tissue.bgef"

with h5py.File(path, "r") as f:
    print("Root attributes:")
    for k, v in f.attrs.items():
        # bytes → 字符串
        if isinstance(v, bytes):
            v = v.decode()
        # numpy 标量 → 提取值但保留 dtype
        if isinstance(v, np.generic):
            print(f"  {k}: {v.item()} (dtype={v.dtype})")
        else:
            print(f"  {k}: {v} (type={type(v).__name__})")

Root attributes:
  bin_type: Bin (type=str)
  gef_area: 0.0 (dtype=float32)
  geftool_ver: [ 1  1 20] (type=ndarray)
  omics: Transcriptomics (type=str)
  sn: Y00855ND (type=str)
  version: 4 (dtype=uint32)


Y00855ND.tissue.gem.gz文件总行数是123196419

In [9]:
!gzcat Y00855ND.tissue.gem.gz | wc -l 

 123196419


In [9]:
# 代码示例
import h5py
import numpy as np
path = "/Volumes/MacPassport/project/bioinformatics_project/MCAO20250929/data/test.bgef"
path = "dummy.bgef"
with h5py.File(path, "r") as f:
    # h5path = "/geneExp/bin1/expression"
    h5path = "/wholeExpExon/bin1"

    # 输出属性
    print(">>>Attribute:")
    for k, v in f[h5path].attrs.items():
        # 一些属性是 bytes，需要手动 decode
        if isinstance(v, bytes):
            v = v.decode()
        print(f"{k}: {v}, {v.dtype}")
    
    # 数据矩阵
    print(">>>Dataset:")
    expr = np.array(f[h5path]) # 替换这里f的键值即可
    print(expr.shape, expr)

>>>Attribute:
maxExon: 28, uint32
>>>Dataset:
(17502, 17043) [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
